# TOPOS Data Engineer Intern Assignment by Viral Pandey

### ASSIGNMENT :  Write a scraper in either python or NodeJS to collect data from Wikipedia about the top cities in the United States. The fields you collect, as well as the volume of data is up to you, but ideally you add additional data beyond the initial table, such as data found on the individual city pages, or other sources of your choice. The final format should be a CSV file that is ready to be uploaded to a BigQuery table. Please read Bigquery’s Manual to prepare your CSV in the right format. Intermediary steps, environments or processes necessary to run the scraper should be documented in code as well as a Readme.md and hosted on github in a repo devoted to this assignment. 

This is a Python Notebook for this assignment where I have used Beautiful Soup to scrape the List of top US cities by population from Wikipedia.

In [244]:
## installing necessary packages for this assignement if not present in the system
!pip install pandas
!pip install numpy
!pip install beautifulsoup4
!pip install requests

In [245]:
## Following packages are needed for this assignment
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [246]:
'''
This function scrapes data from the main table of top cities of US by population 
from Wikipedia. 
It returns a pandas DataFrame
'''
def get_main_table():
    url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
    list_b = BeautifulSoup(requests.get(url).text, features="lxml")
    wiki_tables = list_b.find_all("table", {"class": "wikitable"})
    citylist_table = wiki_tables[1]
    list_rows = citylist_table.find("tbody").find_all("tr")[1:]
    list_headers = ["Rank", "City", "State", "2018 Estimate", "2010 Census", 
                         "Change", "2016 Land Area Km", "2016 Land Area Mi", 
                         "2016 Population Density km", "2016 Population Density mi"]
    city_list = []

    for i, city in enumerate(list_rows):
        cities_data = city.find_all("td")
        city_dict = {}
        for index, city_data in enumerate(cities_data):
            if index < 10:
                if index == 1:
                    city_data = city_data.find("a")
                    city_dict[list_headers[index]] = city_data.text.replace("\n", "")
                    city_dict["Wiki link"] = city_data['href']
                else:
                    city_dict[list_headers[index]] = city_data.text.replace("\n", "")

        if city_dict:
            city_list.append(city_dict)

    city_df = pd.DataFrame(city_list)
    city_df["Land Area"] = np.NaN
    city_df["Water Area"] = np.NaN
    city_df['Climate'] = np.NaN
    city_df['Mayor'] = np.NaN
    city_df['Time Zone'] = np.NaN
    city_df['Airport'] = np.NaN
    city_df['Official Website'] = np.NaN

    return city_df

In [247]:
'''
This function scrapes data from the individual cities mentioned in the main tablelist 
and scrapes additional information of it by going through the its wiki link. 
It takes input of a city as a pandas series object, adds additional information and then 
returns it's pandas series object
'''
def get_city_data(city):
    city_b = BeautifulSoup(requests.get("https://en.wikipedia.org{}".format(city["Wiki link"])).text, 
                              features="lxml")
    gen_info = city_b.find("table", {"class": "infobox geography vcard"})
    
    #Time Zone
    time_zone_data = gen_info.find("a", text = "Time zone")
    time_zone = time_zone_data.find_next("td").text if time_zone_data else None
    if time_zone:
        city["Time Zone"] = time_zone
    
    #Official Website
    official_web_data = gen_info.find("th", text="Website")
    official_web = official_web_data.find_next("a", attrs={'href': re.compile("^http://")}) if official_web_data else None
    official_web = official_web.get('href') if official_web_data else None
    if official_web:
        city["Official Website"] = official_web
    
    #Land Area    
    land_area_data = gen_info.find("a", text="Land")
    if not land_area_data:
        try:
            land_area_data = gen_info.find_all("th", string=re.compile("Land"))[0]
        except:
            land_area_data = None
    land_area = land_area_data.find_next("td").text if land_area_data else None
    if land_area:
        city["Land Area"] = land_area
        
    # Water Area
    water_area_data = gen_info.find("a", text="Water")
    if not water_area_data:
        try:
            water_area_data = gen_info.find_all("th", string=re.compile("Water"))[0]
        except:
            water_area_data = None
    water_area = water_area_data.find_next("td").text if water_area_data else None
    if water_area:
        city["Water Area"] = water_area

    #Mayor
    mayor_data = gen_info.find("a", text="Mayor")
    if not mayor_data:
        try:
            mayor_data = gen_info.find_all("th", string=re.compile("Mayor"))[0]
        except:
            mayor_data = None
    mayor = mayor_data.find_next("a").text if mayor_data else None
    if mayor:
        city["Mayor"] = mayor
    
    #Climate
    climate_data = city_b.find('span', id='Climate')
    if not climate_data:
        climate_data = city_b.find('span', id="Weather")        
    if climate_data:
        climate_data = climate_data.parent.find_next("p").text
        if climate_data:
            city["Climate"] = climate_data.replace("\xa0", "")
    
    #Airports
    airport_data = city_b.find('span', id='Airports')
    if not airport_data:
        airport_data = city_b.find('span', id="Air")            
    if airport_data:
        airport = airport_data.parent.find_next("p")
        
        if airport:
            airport = airport.text
            city["Airport"] = airport.replace("\xa0", "")

    return city

In [248]:
'''
This is a main function that first scrapes the main table from the wikipedia link. 
Then it runs a for loop and fills extra details
It returns a final pandas DataFrame required by this assignment
'''
def top_cities_US():
    city_df = get_main_table()

    for i, city in city_df.iterrows():
        city = get_city_data(city)
        city_df.loc[i] = city

    return city_df

In [250]:
top_cities_US_df = top_cities_US()

In [251]:
'''
Rearrange the columns according to the table on the Wikipedia Page
'''
cols = top_cities_US_df.columns.tolist()
cols = ["Rank", "City", "State", "2018 Estimate", "2010 Census", 
        "Change", "2016 Land Area Km", "2016 Land Area Mi", 
        "2016 Population Density km", "2016 Population Density mi", 
        "Land Area", "Water Area", "Time Zone", "Mayor","Airport",
        "Official Website", "Climate", "Wiki link"]
top_cities_US_df = top_cities_US_df[cols] 

In [252]:
'''
Convert the Pandas DataFrame into a csv file and write it to the local machine
'''
f = open("top_cities_US.csv", "w")
f.write(top_cities_US_df.to_csv(index = False))
f.close()